# 用于测试功能的代码

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.1.2+cu118
True


In [3]:
i=1
print(f"ok{i}")


ok1


当前进度：100.00%

In [1]:
from bs4 import BeautifulSoup
import requests

def ask_url(url):
    head = {    
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47"
        # "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"
    }
    try:
        r = requests.get(url, headers=head, timeout=30)
        r.raise_for_status()
        r.encoding = 'utf-8'
        return r.text
    except:
        return ""
    
    
for i in range(0, 1):
            url = "https://jbk.39.net/bw/" + f"p{i + 1}/"
            html = ask_url(url)
            if html == "":
                continue
            soup = BeautifulSoup(html, 'html.parser')
        
            # 遍历每一种疾病
            for item in soup.find_all('div', class_="result_item"):
                if item.div.p.span.string == "疾病":
                    # 疾病url
                    disease_url = item.div.p.a.attrs["href"]
                    
                    # 疾病简介
                    disease_jianjie_url = disease_url + "jbzs/" 
                    jianjie_html = ask_url(disease_jianjie_url)
                    if jianjie_html == "":
                        continue
                    jianjie_soup = BeautifulSoup(jianjie_html, 'html.parser')
                    name = jianjie_soup.find('div', class_="disease").h1.string
                    # print(name)
                    jianjie = jianjie_soup.find('p', class_="introduction").string
                    # print(jianjie)
                    for disease_basic in jianjie_soup.find_all('ul', class_="disease_basic"):
                        for li in disease_basic.find_all('li'):
                            if li.span.string == "就诊科室：":
                                span = li.find_all('span')[1]
                                category = ""
                                for a in span.find_all('a'):
                                    category += a.string + ","
                                    department_url = a.attrs["href"]
                                    department_html = ask_url(department_url)
                                    if department_html == "":
                                        continue
                                    department_soup = BeautifulSoup(department_html, 'html.parser')
                                category = category[:-1]
                                # print(category)    
                            elif li.span.string == "相关症状：":
                                span = li.find_all('span')[1]
                                symptom = []
                                for a in span.find_all('a'):
                                    symptom_name = a.string
                                    symptom.append(symptom_name)
                                    
                                    
                                # symptom = symptom[:-1]
                                print(symptom) 
                                
                            
                           
                    
                    

['器质性早泄', '男子性功能障碍', '心理性性功能障碍', '遗精滑泄', '详细']
['湿疣', '外阴鳞状上皮疣状增生', '出血性丘疹', '溃疡分泌物恶臭', '乳头状肿物', '详细']
['椎间盘退行性变', '腰脊椎疼痛', '惧站立，喜依托', '胸腰段及腰椎前凸消失', '根性坐骨神经痛', '详细']
['静脉曲张性外痔', '痔出血', '便血鲜红', '排便障碍', '炎性外痔', '详细']
['心理性性功能障碍', '男子性功能障碍', '雄激素过少', '精冷不育', '脉涩或结', '详细']
['乳腺隐痛', '结节', '囊肿', '经期前乳房痛', '腋窝痛', '详细']
['劳动耐力下降', '胸骨后疼痛', '胸闷', '劳累后心悸', '阵发性夜间呼吸困难', '详细']
['转氨酶增高', '肝肿大', '乙肝表面抗原（HBsAg）阳性', '乙肝e抗原（HBeAg）阳性', '肝功能异常', '详细']
['不明原因发热', '免疫力降低', 'HIV感染', '反复感染', '关节疼痛', '详细']
['剧痒', '风团', '皮肤有压痕', '斑丘疹', '丘疹', '详细']
['第1跖趾关节疼痛和肿大', '关节疼痛', '关节肿胀', '关节畸形', '尿酸盐在关节内沉积增多', '详细']
['腰部钝痛', '肾盏结石', '肾区叩击痛', '排出结石', '腰部包块', '详细']
['胃酸过多', '食欲较差', '腹胀嗳腐', '胃胀', '胃气上逆', '详细']
['口腔粘膜弥漫充血', '口舌生疮', '口腔痛', '溃疡', '容易上火', '详细']
['阴道分泌物增多', '排尿时外阴灼热、瘙痒或疼痛', '性交疼痛', '阴部灼热', '白带增多', '详细']


In [6]:

str = "1,2,3,4,5,"
list = str.split(",")
print(list)


['1', '2', '3', '4', '5', '']


In [ ]:
for item,index in enumerate(list):
    print(item,index)

In [2]:
import pymongo
conn = pymongo.MongoClient('localhost', 27017) # 连接数据库
db = conn['medintel'] # 建立数据库
col = db['test'] # 建表，字典形式

data = {
    'name': '张三',
    'age': 18,
}
col.insert_one(data) # 插入数据
print('插入成功')

插入成功


In [3]:
for item in range(0, 10):
    print(item)

0
1
2
3
4
5
6
7
8
9


In [11]:
with open("dataset/data/test.txt", "r") as f:
    lines = f.readlines()
    if 'asdsd\n' in lines:
        print("asdsd is in the list")
    else:
        print("asdsd is not in the list")

asdsd is in the list


In [1]:
import pymongo

conn = pymongo.MongoClient('localhost', 27017) 
db = conn['medintel'] 
col = db['data'] 
data = col.find()


with open("dataset/data/test.txt", "w") as f:
    for item in data:
        f.write(str(item["diseases"]["name"]) + "\n")

In [7]:
my_set = set()
my_set.add(1)
my_set.add(2)
my_set.add(1)

print(my_set)


{1, 2}


In [14]:
list = [1,2,3,1,5,6,4,3,51,2,1,]
set = set(list)
set

TypeError: 'set' object is not callable

In [2]:
disease = set([1,23,15,5,1,65156151,1,15,16,56,165,1,56156,515,1,12,123])
# 定义集合与文件的映射关系
data_to_file = {
    "diseases": ("dataset/data/test.txt", disease)
}

# 遍历映射关系，将数据写入文件
for name, (filename, data) in data_to_file.items():
    print("start")
    with open(filename, "a", encoding="utf-8") as file:  # 使用追加模式
        for item in data:
            file.write(str(item) + "\n")  # 每个数据占一行
    print(f"已写入 {len(data)} 条数据到 {filename}")

start
已写入 12 条数据到 dataset/data/test.txt


In [ ]:
from torch import nn
from torchvision import models

m_model = models.resnet18()
m_model.fc = nn.Linear(512, 10)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
print(1e-4)

0.0001
